# 遗传算法

_本项目所有代码，结果数据，图片等均已开源，遵循MIT开源协议 Lumozz https://github.com/Lumozz/Genetic_Algonrithm_CartPole-v1_

- 本项目将通过尽可能直观的方式展示遗传算法(Genetic Algonrithm)的基本思想与计算流程，因此，本项目将重点放在了思想与流程上，对于其中的数学证明细节，本项目不做深究。
- 遗传算法最早由J.D.Bagley在1967年提出，经过了半个世纪的发展，如今已经成为了一种成熟的具有极高鲁棒性和广泛适用性的全局优化算法。这一算法仿照了真实生物的繁衍方式，即物竞天择，适者生存。
- 中学生物课堂教给了我们基础的生物遗传过程，这一过程大致可以分为三个部分：选择、交叉、变异。  


    - 选择是指从群体中选择优良的个体并淘汰劣质个体的操作。
    - 交叉是指把两个父代个体的部分结构加以替换重组，生成新的个体。
    - 变异就是以很小的概率随机地改变群体中某些个体的染色体的某些基因。  
    
在以下示例中，我们将构造一个机器人群体，设计他们的基因结构，然后让他们按照自然界生物的进化方式，淘汰弱者，孕育子代，不断往复循环，最终得到一个高度进化的机器人群体。

## 导入依赖包与环境

gym包是openAI公司的开源项目，该项目提供了一系列的环境，以支持机器学习，尤其是强化学习的发展。
在本项目中，我们使用gym的“**CartPole-v1**”环境，这个环境可以看作一个小游戏，有一个黑色的滑块可以受控制地左右移动，滑块上立着一根杆子，杆子的一端连在滑块上，可以自由旋转。我们要做的事就是控制滑块的左右移动，让上方的杆子保持直立，保持的时间越长，得分越高。一旦杆子倒下超过15°，视为游戏结束。

先来看一下界面是什么样子的  
<img src="pic/00.png" width = "400" height = "300" alt="pic" align=center />

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('CartPole-v1')
env.seed(1)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


E:\anaconda\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


[1]

## 设置种群个体数量

一个种群中不可能只有一个个体，为了让这个机器人种群中的基因更加丰富，在此，我们设置一万个初始机器人。这些机器人是所有后续机器人的祖先。

In [3]:
number_of_robots = 10000

## 定义基因

我们之前设置的“立杆子”的小游戏，可以看作机器人种群生存的环境，他们从出生那一刻起，就要去立杆子，机器人可以观察到滑块的位置、速度，杆子的位置、角速度四项信息，而它们的可以根据观察到的信息做出动作，动作很简单，只有向左或者向右，可以用一个数值来表示，0表示向左，1表示向右。

In [4]:
print(env.observation_space,env.action_space)

Box(4,) Discrete(2)


要把四个数值的观察值映射到一个数值的动作值，神经网络函数再适合不过了。
所谓神经网络，实际上就是一堆一元一次函数的组合，组合的方式就构成了神经网络的结构，在固定的结构下，每个函数中的参数值就决定了一定的输入会得到什么样的输出。
因此，我们把神经网络中的参数值看作机器人的“基因”。
在初始状态下，种群中一万个机器人的基因都是随机数，我们来定义一个robot_gene类，里面随机初始化了神经了网络各层的参数，在直观上，我们将它看作随机的初始化基因

In [5]:
class robot_gene():
    def __init__(self, number_of_robots=1000):
        self.weight1 = np.random.randn(number_of_robots, 6, 4)
        self.bais1 = np.random.randn(number_of_robots, 6)
        self.weight2 = np.random.randn(number_of_robots, 1, 6)
        self.bais2 = np.random.randn(number_of_robots, 1)
        self.rewards = np.zeros(number_of_robots)

        self.number_of_robots = number_of_robots

    def next_generation(self):
        self.weight1 = np.zeros((self.number_of_robots, 6, 4))
        self.bais1 = np.zeros((self.number_of_robots, 6))
        self.weight2 = np.zeros((self.number_of_robots, 1, 6))
        self.bais2 = np.zeros((self.number_of_robots, 1))
        self.rewards = np.zeros((self.number_of_robots))
    def load(self):
        self.weight1 = np.load('gene.weight1.npy')
        self.bais1 = np.load('gene.bais1.npy')
        self.weight2 = np.load('gene.weight2.npy')
        self.bais2 = np.load('gene.bais2.npy')

机器人单有基因是没有用的，就像人一样，我们不仅需要基因，还需要直到如何表达基因。
对于机器人来说，在当前的状态下（观察值给出），我们需要计算出机器人的动作是什么。从数学的角度来讲，我们要根据输入（观察值）计算出神经网络的输出。
神经网络只有一层中间层，在这层中间层后，我们使用一个sigmoid损失函数。
在输出层，输出的是一个数，如果这个数小于0.5，我们直接把它设置为0，如果大于等于0.5，则设置为1。

robot_gene类中，定义了神经网络结构如下： 
<img src="pic/nn.png" width = "400" height = "300" alt="pic" align=center />
其中：
<center>$X_{1} = g(W_{0}^{T}X_{0}+B_{0})$ </center> 

<center>  </center>  
<center>$Y = W_{1}^{T}X_{1}+B_{1}$ </center>    

<center>  </center>

<center>$g = \frac{1}{(1+e^{-X})}$</center>


In [6]:
def calculate(observation,gene,index):
    x1 = gene.weight1[index].dot(observation.T)+gene.bais1[index]
    x2 = 1/(1 + np.exp(-x1))
    x3 = gene.weight2[index].dot(x2.T)+gene.bais2[index]
    y = np.where(x3 > 0.5, 1, 0)
    return y

- 现在，机器人们已经可以自己玩游戏了，根据观察到的情况，做出相应的反应。但是，他们可以做，不意味着他们能做得好。  
- 很显然，机器人在立杆子游戏中的表现完全是由它们的基因决定的（这很容易理解，因为由观察值到动作是我们算出来的），如果一个机器人表现的不好，就说明它的基因不够优秀，按照自然界的的法则，不够优秀的基因大概率是要被淘汰的（没有机会繁衍后代）。  
- 为了模仿自然选择的过程，我们可以设计一套很残酷的淘汰规则，在这一万个个体当中，表现最好的前一半可以继续存活，并且繁育后代，而后一半将被“杀死”，永远消失。在遗传算法中，并不是表现差的个体一定会被淘汰，也不是表现最好的个体就一定会被保留，淘汰这个过程被看作概率随机过程，表现好的个体有更高的概率存活，表现越好，概率越高，这样的目的也是为了增加随即搜索的宽度和深度。但是在本项目中，为了简便，我们直接采取这种残酷的一刀切的方式筛选优良基因。

## 淘汰

让我们看看最开始的这一万个机器人表现怎么样

In [7]:
gene=robot_gene(number_of_robots)
for index_of_robot in range(0, number_of_robots):
        observation = env.reset()
        ep_reward = 0
        for step in range(0, 10000):  # Don't infinite loop while learning
            behavior = calculate(observation, gene, index_of_robot)
            observation, reward, done, _ = env.step(behavior.item())
            ep_reward = ep_reward + reward
            if done:
                break
        gene.rewards[index_of_robot] = ep_reward

In [8]:
print('最长时间：',gene.rewards.max()/100,'\n','最短时间：',gene.rewards.min()/100,'\n',\
     '平均时间：',gene.rewards.mean()/100)

最长时间： 5.0 
 最短时间： 0.08 
 平均时间： 0.122612


- 这个立杆子游戏每局最长时间被限定在了5秒，从第一次实验来看，某些机器人天资聪颖，从一开始就能坚持满五秒，而大多数的机器人连一秒都坚持不到，因此平均坚持时间只有0.12秒。坚持时间最短的那个只有0.08秒。
- 现实很残酷，按照优胜劣汰的法则，为了让整个种群能够进化，只能选择剔除弱者。这里把得分较低的五千个机器人“杀死”，剩下个五千个得分高的。

In [9]:
def disuse(gene):
    next_generation = robot_gene(number_of_robots)
    next_generation.next_generation()
    several_robot_index = np.argpartition(gene.rewards, -int(number_of_robots/2))[-int(number_of_robots/2):]
    for i,index_of_robot in zip(range(0,len(several_robot_index)),several_robot_index):
        next_generation.weight1[i] = gene.weight1[index_of_robot]
        next_generation.bais1[i] = gene.bais1[index_of_robot]
        next_generation.weight2[i] = gene.weight2[index_of_robot]
        next_generation.bais2[i] = gene.bais2[index_of_robot]
    return next_generation

按照遗传算法理论，并不是弱者一定会被淘汰，也不是强者一定会生存下去，一个个体是否会被淘汰被看作一个概率事件，但是在这里为了简化算法，选择一刀切的方式。

## 繁育后代

进行了一轮淘汰后，种群中只剩下了5000个机器人个体。如同孩子的染色体来自于他的父母，我们同样希望机器人子代的基因来自于两个父代机器人。
不要忘了，机器人的基因是神经网络函数中的参数。我们只要取某一个个体基因的一半，与另一个个体的基因的一半组合起来，就相当于这两个个体生成了一个子代个体。具体来说，这里将前2500个个体基因与后2500个个体的基因组合，生成了5000个新的个体。

In [10]:
def cross_swapa(next_generation):
    next_generation.weight1[int(number_of_robots / 2):int(number_of_robots * 3 / 4), :, 0:2] = next_generation.weight1[
                                                                                               0:int(
                                                                                                   number_of_robots / 4),
                                                                                               :, 0:2].copy()
    next_generation.weight1[int(number_of_robots * 3 / 4):int(number_of_robots), :, 0:2] = next_generation.weight1[
                                                                                           int(number_of_robots / 4):int(
                                                                                               number_of_robots / 2), :,
                                                                                           0:2].copy()

    next_generation.weight1[int(number_of_robots / 2):int(number_of_robots * 3 / 4), :, 2:4] = next_generation.weight1[
                                                                                               int(number_of_robots / 4):int(
                                                                                                   number_of_robots / 2),
                                                                                               :, 2:4].copy()
    next_generation.weight1[int(number_of_robots * 3 / 4):int(number_of_robots), :, 2:4] = next_generation.weight1[
                                                                                           0:int(number_of_robots / 4),
                                                                                           :, 2:4].copy()

    next_generation.weight2[int(number_of_robots / 2):int(number_of_robots * 3 / 4), :, 0:3] = next_generation.weight2[
                                                                                               0:int(
                                                                                                   number_of_robots / 4),
                                                                                               :, 0:3].copy()
    next_generation.weight2[int(number_of_robots * 3 / 4):int(number_of_robots), :, 0:3] = next_generation.weight2[
                                                                                           int(number_of_robots / 4):int(
                                                                                               number_of_robots / 2), :,
                                                                                           0:3].copy()

    next_generation.weight2[int(number_of_robots / 2):int(number_of_robots * 3 / 4), :, 3:6] = next_generation.weight2[
                                                                                               int(number_of_robots / 4):int(
                                                                                                   number_of_robots / 2),
                                                                                               :, 3:6].copy()
    next_generation.weight2[int(number_of_robots * 3 / 4):int(number_of_robots), :, 3:6] = next_generation.weight2[
                                                                                           0:int(number_of_robots / 4),
                                                                                           :, 3:6].copy()

    next_generation.bais1[int(number_of_robots / 2):int(number_of_robots * 3 / 4), 0:3] = next_generation.bais1[
                                                                                          0:int(number_of_robots / 4),
                                                                                          0:3].copy()
    next_generation.bais1[int(number_of_robots * 3 / 4):int(number_of_robots), 0:3] = next_generation.bais1[
                                                                                      int(number_of_robots / 4):int(
                                                                                          number_of_robots / 2),
                                                                                      0:3].copy()

    next_generation.bais1[int(number_of_robots / 2):int(number_of_robots * 3 / 4), 3:6] = next_generation.bais1[
                                                                                          int(number_of_robots / 4):int(
                                                                                              number_of_robots / 2),
                                                                                          3:6].copy()
    next_generation.bais1[int(number_of_robots * 3 / 4):int(number_of_robots), 3:6] = next_generation.bais1[
                                                                                      0:int(number_of_robots / 4),
                                                                                      3:6].copy()

这样一来，种群中有初代的五千个个体，子代五千个个体，总数保持了一万没有变。

## 变异

如同人类的基因总会因为各种原因产生一些变异一样，机器人种群也需要变异。变异本身是一种局部随即搜索，与选择、交叉算子结合在一起，就能避免由于选择和交叉计算引起的某些信息的永久丢失。
在本项目中，我们选择了30%的变异概率，即30%的基因会产生变异，这些被选中的基因将加上一个（-2，2）之间均匀分布的随机数。

In [11]:
def mutation(next_generation):
    mutation_index1 = np.random.uniform(low=0, high=1, size=(int(number_of_robots / 2), 6, 4))
    mutation_index2 = np.random.uniform(low=0, high=1, size=(int(number_of_robots / 2), 1, 6))
    mutation_index3 = np.random.uniform(low=0, high=1, size=(int(number_of_robots / 2), 6))
    mutation_index4 = np.random.uniform(low=0, high=1, size=(int(number_of_robots / 2), 1))

    mutation_index11 = np.random.uniform(low=-2, high=2, size=(int(number_of_robots / 2), 6, 4))
    mutation_index22 = np.random.uniform(low=-2, high=2, size=(int(number_of_robots / 2), 1, 6))
    mutation_index33 = np.random.uniform(low=-2, high=2, size=(int(number_of_robots / 2), 6))
    mutation_index44 = np.random.uniform(low=-2, high=2, size=(int(number_of_robots / 2), 1))

    mutation1 = np.where(mutation_index1 > 0.7, 1, 0)
    mutation1 = mutation1 * mutation_index11
    mutation2 = np.where(mutation_index2 > 0.7, 1, 0)
    mutation2 = mutation2 * mutation_index22
    mutation3 = np.where(mutation_index3 > 0.7, 1, 0)
    mutation3 = mutation3 * mutation_index33
    mutation4 = np.where(mutation_index4 > 0.7, 1, 0)
    mutation4 = mutation4 * mutation_index44

    next_generation.weight1[int(number_of_robots / 2):] = next_generation.weight1[
                                                          int(number_of_robots / 2):] + mutation1
    next_generation.weight2[int(number_of_robots / 2):] = next_generation.weight2[
                                                          int(number_of_robots / 2):] + mutation2
    next_generation.bais1[int(number_of_robots / 2):] = next_generation.bais1[int(number_of_robots / 2):] + mutation3
    next_generation.bais2[int(number_of_robots / 2):] = next_generation.bais2[int(number_of_robots / 2):] + mutation4

至此，遗传的所有步骤都已经实现，最后，我们设置一个循环，让种群不断进化。

## 进化

将上述代码套进一个循环里面，设置50次循环，即让机器人繁衍五十代，来看看会有什么变化

In [34]:
gene=robot_gene(number_of_robots)

for i in range(0, 50):
    
    for index_of_robot in range(0, number_of_robots):
        observation = env.reset()
        ep_reward = 0
        for step in range(0, 10000):  # Don't infinite loop while learning
            behavior = calculate(observation, gene, index_of_robot)
            observation, reward, done, _ = env.step(behavior.item())
            #env.render()
            ep_reward = ep_reward + reward
            if done:
                break
        gene.rewards[index_of_robot] = ep_reward
    
    if i % 10 == 0:
        print(i, ':     ', gene.rewards.mean())
    with open('log.txt', 'a') as f:
        f.write(str(gene.rewards.mean())+'\n')    
    
    next_generation = disuse(gene)
    cross_swapa(next_generation)
    mutation(next_generation)

    gene = next_generation

0 :      11.9298
10 :      48.8467
20 :      120.3852
30 :      233.33
40 :      340.8145


In [ ]:
np.save("filename.npy",a)

经过50次迭代后，个体的平均坚持时间已经到了3.4秒，可谓进步明显。机器人迭代五十代需要十几分钟，人类繁衍五十代却需要一千年！  
我们让种群继续迭代150代，看看会发生什么。

<img src="pic/data.png" width = "600" height = "500" alt="pic" align=center />

迭代到第150次时，个体平均坚持时间已经可以达到4秒了。观察曲线，可得知遗传算法起了作用。

## 测试

继续迭代，经过五百次迭代后，看看种群整体会有什么表现。  
加载保存的数据

In [39]:
gene=robot_gene(number_of_robots)
gene.load()
for index_of_robot in range(0, number_of_robots):
        observation = env.reset()
        ep_reward = 0
        for step in range(0, 10000):  # Don't infinite loop while learning
            behavior = calculate(observation, gene, index_of_robot)
            observation, reward, done, _ = env.step(behavior.item())
            #env.render()
            ep_reward = ep_reward + reward
            if done:
                break
        gene.rewards[index_of_robot] = ep_reward

In [40]:
print('最长时间：',gene.rewards.max()/100,'\n','最短时间：',gene.rewards.min()/100,'\n',\
     '平均时间：',gene.rewards.mean()/100)

最长时间： 5.0 
 最短时间： 0.08 
 平均时间： 4.24463


种群平均坚持时间已经达到了4.24秒，这是一个相当高的数字了。不过，仍然有些个体在拖后腿。